In [1]:
import numpy as np
from scipy import stats
import torch

# Testing of the generated models.

## Generating the test-data.

### `autoregressive_moving_average_model`

In [2]:
data_length = 20
mu = 25
phi = 3
theta = 1.5
sigma = 3

In [3]:
np.random.seed(0)
err = torch.full((data_length,), 0, dtype=float)
for t in range(0, data_length, 1):
    err[t] = stats.norm.rvs(10, 5)
err

tensor([18.8203, 12.0008, 14.8937, 21.2045, 19.3378,  5.1136, 14.7504,  9.2432,
         9.4839, 12.0530, 10.7202, 17.2714, 13.8052, 10.6084, 12.2193, 11.6684,
        17.4704,  8.9742, 11.5653,  5.7295], dtype=torch.float64)

In [4]:
np.random.seed(0)
y = torch.full((data_length,), 0, dtype=float)
nu = torch.full((data_length,), 0, dtype=float)
nu[0] = mu + phi * mu
y[0] = err[0] + nu[0]
for t in range(1, data_length):
    nu[t] = mu + phi * y[t - 1] + theta * err[t - 1]
    y[t] = err[t] + nu[t]
y

tensor([1.1882e+02, 4.2169e+02, 1.3230e+03, 4.0375e+03, 1.2189e+04, 3.6625e+04,
        1.0992e+05, 3.2982e+05, 9.8951e+05, 2.9686e+06, 8.9058e+06, 2.6717e+07,
        8.0152e+07, 2.4046e+08, 7.2137e+08, 2.1641e+09, 6.4924e+09, 1.9477e+10,
        5.8431e+10, 1.7529e+11], dtype=torch.float64)

### `autoregressive_order_K_model`

In [5]:
K = 5
alpha = 14
beta = [-6.45,6.93,-2.48,-1.99,12.19]
sigma = 0.5

In [6]:
np.random.seed(0)
y = torch.zeros(20)
for t in range(K, len(y)):
    mu = alpha
    for k in range(0, K):
        mu = mu + beta[k] * y[t-k]
    y[t] = stats.norm.rvs(mu, sigma)
y

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.4882e+01,
        1.1733e+02, 7.9070e+02, 5.1740e+03, 3.3858e+04, 2.2168e+05, 1.4516e+06,
        9.5055e+06, 6.2245e+07, 4.0760e+08, 2.6691e+09, 1.7478e+10, 1.1445e+11,
        7.4944e+11, 4.9076e+12])

### `bayes_hidden_markov_model`

In [7]:
K = 10
s = [0] * K
m = [0] * K
T = [[0] * K] * K

In [8]:
np.random.seed(0)
for i in range(0, K):
    T[i] = stats.dirichlet.rvs([1 / K] * K)
    m[i] = stats.norm.rvs(i + 1, 0.5).item()

In [9]:
np.random.seed(0)
y = [0] * K
s[0] = stats.randint.rvs(0, K)
y[0] = stats.norm.rvs(m[s[0]], 0.1).item()
for i in range(1, K):
    s[i] = stats.randint.rvs(0, K)
    y[i] = stats.norm.rvs(m[s[i]], 0.1).item()
s, y

([5, 7, 9, 7, 6, 1, 6, 1, 5, 4],
 [6.500169164315597,
  7.936579509333255,
  9.317801661471384,
  8.077189678856062,
  6.647212557828355,
  1.8866898228924214,
  6.6439487634103465,
  2.0341088048097773,
  6.441242417801613,
  5.201439088726868])

### `burglary_model`

In [10]:
earthquake_probability = 0.02
burglary_probability = 0.01
phone_working_earthquake_probability = 0.8
phone_working_probability = 0.9
mary_wakes_earthquake_probability = 0.7
mary_wakes_burglary_probability = 0.8
mary_wakes_probability = 0.1

In [11]:
np.random.seed(0)
earthquake = stats.bernoulli.rvs(earthquake_probability)
burglary = stats.bernoulli.rvs(burglary_probability)
if earthquake == 1:
    phone_working = stats.bernoulli.rvs(phone_working_earthquake_probability)
else:
    phone_working = stats.bernoulli.rvs(phone_working_probability)
if earthquake == 1:
    mary_wakes = stats.bernoulli.rvs(mary_wakes_earthquake_probability)
elif burglary == 1:
    mary_wakes = stats.bernoulli.rvs(mary_wakes_burglary_probability)
else:
    mary_wakes = stats.bernoulli.rvs(mary_wakes_probability)
called = mary_wakes == 1 and phone_working == 1
called

False

### `cointoss_model` and `cointoss_with_factor_model`

In [12]:
p = 0.1

In [13]:
np.random.seed(0)
tosses = [stats.bernoulli.rvs(p) for _ in range(20)]
tosses

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

### `gaussian_mixture_model`

In [14]:
theta = 0.3
mu = [-2.4,1.2]
z = [0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0]

In [15]:
np.random.seed(0)
points = [stats.norm.rvs(mu[z[i]], 10).item() for i in range(len(z))]
points

[15.24052345967664,
 5.201572083672233,
 7.387379841057392,
 23.60893199201458,
 19.875579901499673,
 -8.57277879876411,
 7.100884175255894,
 -3.913572082976979,
 0.16781148206442142,
 5.305985019383724,
 -0.9595642883912201,
 12.142735069629751,
 8.810377251469934,
 2.416750164928284,
 5.638632327454257,
 4.536743273742668,
 16.14079073157606,
 -0.8515826376580089,
 0.7306770165090137,
 -10.940957393017248]

### `linear_regression_model`

In [16]:
slope = 0.5
intercept = 1

In [17]:
np.random.seed(0)
xs = torch.tensor([i+1 for i in range(10)])
ys = torch.tensor([stats.norm.rvs(slope * x + intercept, 0.2) for x in xs])
xs, ys

(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 tensor([1.8528, 2.0800, 2.6957, 3.4482, 3.8735, 3.8045, 4.6900, 4.9697, 5.4794,
         6.0821], dtype=torch.float64))

### `number_of_heads_model`

In [18]:
p = 0.01

In [19]:
np.random.seed(0)
count = 0
while True:
    cointoss = stats.bernoulli.rvs(p)
    if cointoss == 1:
        break
    count += 1
count

149

### `rate_5_model`

In [20]:
n1 = 50
n2 = 75
theta = 0.6

In [21]:
np.random.seed(0)
k1 = stats.binom.rvs(n1, theta)
k2 = stats.binom.rvs(n2, theta)
k1, k2

(30, 43)